In [2]:
import datetime
import json
import requests

import pandas as pd

from bs4 import BeautifulSoup

### Making your first request

In [3]:
url = 'https://www.gov.br/planalto/pt-br/acompanhe-o-planalto/agenda-do-presidente-da-republica/2021-07-13'
response = requests.get(url)

print("Response Object: ", response)
print("Response Status Code: ", response.status_code)
print("Response URL: ", response.url, "\n\n")
print("Request Headers :", response.request.headers, "\n\n")
print("Response Headers: ", response.headers)

Response Object:  <Response [200]>
Response Status Code:  200
Response URL:  https://www.gov.br/planalto/pt-br/acompanhe-o-planalto/agenda-do-presidente-da-republica/2021-07-13 


Request Headers : {'User-Agent': 'python-requests/2.22.0', 'Accept-Encoding': 'gzip, deflate', 'Accept': '*/*', 'Connection': 'keep-alive'} 


Response Headers:  {'Date': 'Tue, 13 Jul 2021 22:58:54 GMT', 'Server': 'Zope/(2.13.28, python 2.7.16, linux2) ZServer/1.1', 'X-Cache-Operation': 'plone.app.caching.moderateCaching', 'Content-Language': 'pt-br', 'Expires': 'Sat, 16 Jul 2011 22:58:54 GMT', 'Vary': 'X-Anonymous,Accept-Encoding', 'X-Ua-Compatible': 'IE=edge,chrome=1', 'X-Cache-Rule': 'brasil.gov.agenda.AgendaDiaria', 'X-Frame-Options': 'DENY', 'Content-Type': 'text/html;charset=utf-8', 'cache-control': 'max-age=10, s-maxage=0, must-revalidate', 'X-Varnish': '9667783 8856825', 'Via': '1.1 varnish (Varnish/6.0), 1.1 www.gov.br', 'X-Cache': 'HIT', 'X-Varnish-Age': '905', 'Age': '0', 'Accept-Ranges': 'bytes', 

### Retrieving the page's content

In [96]:
#response.content

### Using BeautifulSoup

#### Requesting a single page

In [5]:
start_date = '2021-07-13'
url = 'https://www.gov.br/planalto/pt-br/acompanhe-o-planalto/agenda-do-presidente-da-republica/{sd}'.format(sd=start_date)
response = requests.get(url)

In [6]:
soup = BeautifulSoup(response.content)
compromissos_tags = soup.findAll('li', class_='item-compromisso-wrapper')

In [7]:
compromisso_lista = []
for comp in compromissos_tags:
    compromisso = {}
    compromisso['titulo'] = comp.find('h4', class_='compromisso-titulo').text
    compromisso['start_hour'] = comp.find('time', class_='compromisso-inicio').text
    compromisso['end_hour'] = comp.find('time', class_='compromisso-fim').text
    compromisso['location'] = comp.find('div', class_='compromisso-local').text
    compromisso['reference_date'] = start_date
    compromisso_lista.append(compromisso)

    
compromisso_lista

[{'titulo': 'Luiz Eduardo Ramos, Ministro de Estado Chefe da Casa Civil da Presidência da República',
  'start_hour': '10h00',
  'end_hour': '10h30',
  'location': 'Palácio do Planalto',
  'reference_date': '2021-07-13'},
 {'titulo': 'Senador Eduardo Gomes (MDB/TO), Vice-Líder do Governo no Senado Federal; e Senador Jayme Campos (DEM/MT)',
  'start_hour': '14h30',
  'end_hour': '15h00',
  'location': 'Palácio do Planalto',
  'reference_date': '2021-07-13'},
 {'titulo': 'Paulo Guedes, Ministro de Estado da Economia',
  'start_hour': '15h30',
  'end_hour': '16h00',
  'location': 'Palácio do Planalto',
  'reference_date': '2021-07-13'},
 {'titulo': 'Solenidade alusiva à Sanção da Lei de Capitalização da Eletrobras',
  'start_hour': '16h30',
  'end_hour': '17h30',
  'location': 'Palácio do Planalto',
  'reference_date': '2021-07-13'}]

#### Converting to a pandas DataFrame

In [8]:
df = pd.DataFrame(compromisso_lista)
df

,titulo,start_hour,end_hour,location,reference_date
0,"Luiz Eduardo Ramos, Ministro de Estado Chefe d...",10h00,10h30,Palácio do Planalto,2021-07-13
1,"Senador Eduardo Gomes (MDB/TO), Vice-Líder do ...",14h30,15h00,Palácio do Planalto,2021-07-13
2,"Paulo Guedes, Ministro de Estado da Economia",15h30,16h00,Palácio do Planalto,2021-07-13
3,Solenidade alusiva à Sanção da Lei de Capitali...,16h30,17h30,Palácio do Planalto,2021-07-13


### Requesting multiple pages

In [10]:
START = '2019-01-01'
END = datetime.datetime.today()

start = datetime.datetime.strptime(START, "%Y-%m-%d")
date_generated = [start + datetime.timedelta(days=x) for x in range(0, (END-start).days + 1)]
date_range = [datetime.datetime.strftime(i, "%Y-%m-%d") for i in date_generated]

In [11]:
compromissos_lifetime = []
for d in date_range:
    try:
        url = 'https://www.gov.br/planalto/pt-br/acompanhe-o-planalto/agenda-do-presidente-da-republica/{sd}'.format(sd=d)
        response = requests.get(url)
        soup = BeautifulSoup(response.content)
        compromissos_tags = soup.findAll('li', class_='item-compromisso-wrapper')
        compromisso_lista = []
        for comp in compromissos_tags:
            try:
                compromisso = {}
                title = comp.find('h4', class_='compromisso-titulo')
                compromisso['titulo'] = title.text if title else None
                start_hour = comp.find('time', class_='compromisso-inicio')
                compromisso['start_hour'] = start_hour.text if start_hour else None
                end_hour = comp.find('time', class_='compromisso-fim')
                compromisso['end_hour'] = end_hour.text if end_hour else None
                location = comp.find('div', class_='compromisso-local')
                compromisso['location'] = location.text if location else None
                compromisso['reference_date'] = d
                compromisso_lista.append(compromisso)
                print(compromisso)
            except Exception as e:
                print(e)
        compromissos_lifetime.extend(compromisso_lista)
    except:
        pass


{'titulo': 'Solenidade de Posse', 'start_hour': '15h00', 'end_hour': '18h00', 'location': 'Palácio do Planalto', 'reference_date': '2019-01-01'}
{'titulo': 'Cerimônia de transmissão do cargo de Ministro-Chefe da Casa Civil da Presidência da República, do Senhor Eliseu Padilha ao Senhor Onyx Lorenzoni; do cargo de Ministro da Secretaria-Geral da Presidência, do Senhor Ronaldo Fonseca ao Senhor Gustavo Bebianno; do cargo de Ministro da Secretaria de Governo, do Senhor Carlos Marun ao Senhor General Carlos Alberto; do cargo de Ministro da Segurança Institucional, do Senhor Sergio Etchegoyen ao Senhor General Augusto Heleno', 'start_hour': '09h00', 'end_hour': '10h00', 'location': 'Palácio do Planalto', 'reference_date': '2019-01-02'}
{'titulo': 'Mike Pompeo, Ministro das Relações Exteriores dos EUA', 'start_hour': '10h00', 'end_hour': '10h45', 'location': 'Palácio do Planalto', 'reference_date': '2019-01-02'}
{'titulo': 'Marcelo Rebelo de Sousa, Presidente de Portugal', 'start_hour': '10h

{'titulo': 'Augusto Heleno, Ministro-Chefe do Gabinete de Segurança Institucional da Presidência da República', 'start_hour': '08h00', 'end_hour': '08h20', 'location': 'Palácio do Planalto', 'reference_date': '2019-01-08'}
{'titulo': 'Marcos Pontes, Ministro de Estado da Ciência, Tecnologia, Inovações e Comunicações', 'start_hour': '08h20', 'end_hour': '08h40', 'location': 'Palácio do Planalto', 'reference_date': '2019-01-08'}
{'titulo': 'Fernando Azevedo, Ministro de Estado da Defesa', 'start_hour': '08h40', 'end_hour': '08h55', 'location': 'Palácio do Planalto', 'reference_date': '2019-01-08'}
{'titulo': 'Ricardo Salles, Ministro de Estado do Meio Ambiente', 'start_hour': '08h55', 'end_hour': '09h05', 'location': 'Palácio do Planalto', 'reference_date': '2019-01-08'}
{'titulo': 'Reunião do Conselho de Governo', 'start_hour': '09h05', 'end_hour': '12h00', 'location': 'Palácio do Planalto', 'reference_date': '2019-01-08'}
{'titulo': 'Almoço em homenagem ao Comandante da Marinha, Almira

{'titulo': 'Sem compromisso oficial', 'start_hour': '00h00', 'end_hour': None, 'location': None, 'reference_date': '2019-01-12'}
{'titulo': 'Sérgio Moro, Ministro de Estado da Justiça e Segurança Pública, Ernesto Araújo, Ministro de Estado das Relações Exteriores; e Augusto Heleno, Ministro-Chefe do Gabinete de Segurança Institucional da Presidência da República', 'start_hour': '11h00', 'end_hour': '11h20', 'location': 'Palácio da Alvorada', 'reference_date': '2019-01-13'}
{'titulo': 'Santos Cruz, Ministro-Chefe da Secretaria de Governo da Presidência da República', 'start_hour': '07h30', 'end_hour': '08h10', 'location': 'Palácio da Alvorada', 'reference_date': '2019-01-14'}
{'titulo': 'Augusto Heleno, Ministro-Chefe do Gabinete de Segurança Institucional da Presidência da República', 'start_hour': '08h20', 'end_hour': '08h40', 'location': 'Palácio do Planalto', 'reference_date': '2019-01-14'}
{'titulo': 'Gustavo Bebianno, Ministro-Chefe da Secretaria-Geral da Presidência da República'

{'titulo': 'Deputada Eleita Bia Kicis (PRP/DF)', 'start_hour': '11h00', 'end_hour': '11h30', 'location': 'Palácio do Planalto', 'reference_date': '2019-01-17'}
{'titulo': 'Coronel Rodrigo Nery, Oficial da Reserva, e Hernando Barbosa, Subtenente da Reserva do BOPE/RJ', 'start_hour': '11h30', 'end_hour': '12h00', 'location': 'Palácio do Planalto', 'reference_date': '2019-01-17'}
{'titulo': 'Tereza Cristina, Ministra de Estado da Agricultura, Pecuária e Abastecimento; Santos Cruz, Ministro-Chefe da Secretaria de Governo da Presidência da República; Augusto Heleno, Ministro-Chefe do Gabinete de Segurança Institucional da Presidência da República', 'start_hour': '12h00', 'end_hour': '13h20', 'location': 'Palácio do Planalto', 'reference_date': '2019-01-17'}
{'titulo': 'Onyx Lorenzoni, Ministro-Chefe da Casa Civil da Presidência da \tRepública, e André Luiz de Almeida, Advogado-Geral da União', 'start_hour': '14h00', 'end_hour': '14h30', 'location': 'Palácio do Planalto', 'reference_date': '

{'titulo': 'Chegada a Brasília', 'start_hour': '04h30', 'end_hour': '04h35', 'location': 'Base Aérea de Brasília', 'reference_date': '2019-01-25'}
{'titulo': 'Hamilton Mourão, Vice-Presidente da República; Onyx Lorenzoni, Ministro-Chefe da Casa Civil da Presidência da República; e Augusto Heleno, Ministro-Chefe do Gabinete de Segurança Institucional da Presidência da República', 'start_hour': '10h00', 'end_hour': '11h30', 'location': 'Palácio do Planalto', 'reference_date': '2019-01-25'}
{'titulo': 'Onyx Lorenzoni, Ministro-Chefe da Casa Civil da Presidência da \tRepública, e Fernando Azevedo, Ministro de Estado da Defesa', 'start_hour': '14h00', 'end_hour': '14h30', 'location': 'Palácio do Planalto', 'reference_date': '2019-01-25'}
{'titulo': 'Santos \tCruz, Ministro-Chefe da Secretaria de Governo da Presidência \tda República; Presidência da República, e Deputado Eleito Major \tVitor Hugo (PSL/GO)', 'start_hour': '15h30', 'end_hour': '16h00', 'location': 'Palácio do Planalto', 'refer

...

{'titulo': 'Partida de Brasília/DF para São Gabriel da Cachoeira/AM', 'start_hour': '08h00', 'end_hour': '10h50', 'location': 'Base Aérea de Brasília', 'reference_date': '2021-05-27'}
{'titulo': 'Ato alusivo à inauguração da ponte Rodrigo e Cibele', 'start_hour': '13h30', 'end_hour': '14h30', 'location': 'São Gabriel da Cachoeira/AM', 'reference_date': '2021-05-27'}
{'titulo': 'Partida de São Gabriel da Cachoeira/AM para Santa Isabel do Rio Negro/AM', 'start_hour': '15h25', 'end_hour': '15h40', 'location': 'São Gabriel da Cachoeira/AM', 'reference_date': '2021-05-27'}
{'titulo': 'Inauguração de módulos de energia solar no 5º Pelotão Especial de Fronteira - PEF', 'start_hour': '08h00', 'end_hour': '08h30', 'location': 'Santa Isabel do Rio Negro/AM', 'reference_date': '2021-05-28'}
{'titulo': 'Partida de São Gabriel da Cachoeira/AM para Brasília/DF', 'start_hour': '09h20', 'end_hour': '14h10', 'location': 'São Gabriel da Cachoeira/AM', 'reference_date': '2021-05-28'}
{'titulo': 'Pedro Ce

{'titulo': 'Luiz Eduardo Ramos, Ministro-Chefe da Casa Civil da Presidência da República; Paulo Guedes, Ministro de Estado da Economia; Onyx Lorenzoni, Ministro de Estado Chefe da Secretaria-Geral da Presidência da República', 'start_hour': '09h30', 'end_hour': '10h00', 'location': 'Palácio do Planalto', 'reference_date': '2021-06-09'}
{'titulo': 'Partida de Brasília/DF para Goianápolis/GO', 'start_hour': '11h30', 'end_hour': '12h10', 'location': 'Base Aérea de Brasília', 'reference_date': '2021-06-09'}
{'titulo': 'Partida de Goianápolis/GO para Anápolis/GO', 'start_hour': '13h40', 'end_hour': '13h50', 'location': 'Goianápolis/GO', 'reference_date': '2021-06-09'}
{'titulo': 'Culto Interdenominacional das Igrejas de Anápolis', 'start_hour': '15h30', 'end_hour': '16h30', 'location': 'Anápolis/GO', 'reference_date': '2021-06-09'}
{'titulo': 'Partida de Anápolis/GO para Brasília/DF', 'start_hour': '17h15', 'end_hour': '17h45', 'location': 'Aeroporto de Anápolis', 'reference_date': '2021-06

{'titulo': 'Partida de Brasília/DF para Marabá/PA', 'start_hour': '09h00', 'end_hour': '10h40', 'location': 'Base Aérea de Brasília', 'reference_date': '2021-06-18'}
{'titulo': 'Cerimônia de Entrega de Títulos de Propriedade Rural no estado do Pará', 'start_hour': '12h00', 'end_hour': '13h00', 'location': 'Marabá/PA', 'reference_date': '2021-06-18'}
{'titulo': 'Partida de Marabá/PA para Novo Repartimento/PA', 'start_hour': '13h40', 'end_hour': '14h10', 'location': 'Aeroporto João Corrêa da Rocha', 'reference_date': '2021-06-18'}
{'titulo': 'Cerimônia de Liberação da Pavimentação de 102 km da Rodovia Transamazônica (BR-230/PA) e Assinatura da Ordem de Serviço para o Início das Obras da Ponte sobre o Rio Xingu', 'start_hour': '14h45', 'end_hour': '15h45', 'location': 'Novo Repartimento/PA', 'reference_date': '2021-06-18'}
{'titulo': 'Partida de Novo Repartimento/PA para Belém/PA', 'start_hour': '16h15', 'end_hour': '18h40', 'location': 'Novo Repartimento/PA', 'reference_date': '2021-06-1

{'titulo': 'Partida de Chapecó/SC para Brasília/DF', 'start_hour': '13h45', 'end_hour': '15h45', 'location': 'Chapecó/SC', 'reference_date': '2021-06-26'}
{'titulo': 'Lançamento do Plano Safra do Banco do Brasil 2021/2022', 'start_hour': '10h00', 'end_hour': '11h30', 'location': 'Brasília/DF', 'reference_date': '2021-06-28'}
{'titulo': 'Luiz Eduardo Ramos, Ministro de Estado Chefe da Casa Civil da Presidência da República; Paulo Guedes, Ministro de Estado da Economia; Wagner Rosário, Ministro de Estado da Controladoria-Geral da União; e Flávia Arruda, Ministra de Estado Chefe da Secretaria de Governo', 'start_hour': '14h00', 'end_hour': '15h00', 'location': 'Palácio do Planalto', 'reference_date': '2021-06-28'}
{'titulo': 'Pedro Cesar Sousa, Subchefe para Assuntos Jurídicos da Secretaria-Geral da Presidência da República', 'start_hour': '15h00', 'end_hour': '15h30', 'location': 'Palácio do Planalto', 'reference_date': '2021-06-28'}
{'titulo': 'Onyx Lorenzoni, Ministro-Chefe da Secretar

{'titulo': 'Hasteamento da Bandeira Nacional', 'start_hour': '08h00', 'end_hour': '08h15', 'location': 'Palácio da Alvorada', 'reference_date': '2021-07-06'}
{'titulo': 'Reunião com Ministros de Estado', 'start_hour': '08h15', 'end_hour': '11h30', 'location': 'Palácio da Alvorada', 'reference_date': '2021-07-06'}
{'titulo': 'Milton Ribeiro, Ministro de Estado da Educação', 'start_hour': '11h50', 'end_hour': '12h50', 'location': 'Palácio do Planalto', 'reference_date': '2021-07-06'}
{'titulo': 'Deputado Cezinha de Madureira (PSD/SP), Vice-Líder do Governo no Congresso Nacional', 'start_hour': '15h55', 'end_hour': '16h10', 'location': 'Palácio do Planalto', 'reference_date': '2021-07-06'}
{'titulo': 'Solenidade de assinatura do Projeto de Lei que regulamenta a Profissão de Educação Física', 'start_hour': '16h40', 'end_hour': '17h00', 'location': 'Palácio do Planalto', 'reference_date': '2021-07-06'}
{'titulo': 'Pastor Silas Malafaia, Presidente do Conselho Interdenominacional de Ministro

In [12]:
df = pd.DataFrame(compromissos_lifetime)
df.count()

titulo            4727
start_hour        4727
end_hour          4569
location          4695
reference_date    4727
dtype: int64

In [13]:
df.to_csv('data/agenda_oficial.csv', sep=',', index=False)

In [15]:
dfg = pd.read_csv('data/agenda_oficial.csv', sep=',')
dfg

,titulo,start_hour,end_hour,location,reference_date
0,Solenidade de Posse,15h00,18h00,Palácio do Planalto,2019-01-01
1,Cerimônia de transmissão do cargo de Ministro-...,09h00,10h00,Palácio do Planalto,2019-01-02
2,"Mike Pompeo, Ministro das Relações Exteriores ...",10h00,10h45,Palácio do Planalto,2019-01-02
3,"Marcelo Rebelo de Sousa, Presidente de Portugal",10h45,11h15,Palácio do Planalto,2019-01-02
4,"Viktor Orban, Primeiro Ministro da Hungria",11h15,12h00,Palácio do Planalto,2019-01-02
...,...,...,...,...,...
4722,"Luiz Fux, Presidente do Supremo Tribunal Federal",17h00,17h30,Brasília/DF,2021-07-12
4723,"Luiz Eduardo Ramos, Ministro de Estado Chefe d...",10h00,10h30,Palácio do Planalto,2021-07-13
4724,"Senador Eduardo Gomes (MDB/TO), Vice-Líder do ...",14h30,15h00,Palácio do Planalto,2021-07-13
4725,"Paulo Guedes, Ministro de Estado da Economia",15h30,16h00,Palácio do Planalto,2021-07-13
